In [174]:
type register = 
| A
| B
| C
| D

type expression = 
| Register of register
| Const of int

type address = 
| Address of int

type instruction =
  | MOV of register * expression
  | ADD of register * expression
  | SUB of register * expression
  | INC of register
  | DEC of register
  | MUL of expression
  | DIV of expression
  | AND of register * expression
  | OR of register * expression
  | XOR of register * expression
  | NOT of register
  | CMP of register * expression
  | JMP of address
  | JA of address
  | JAE of address
  | JB of address
  | JBE of address
  | JE of address
  | JNE of address
  | CALL of address
  | RET
  | PUSH of expression
  | POP of register
  | HLT

type state = {
  instructions: instruction array;
  a : int; 
  b : int; 
  c : int; 
  d : int;
  ip : address;
  zero : bool;
  carry : bool;
  stack : int list
  }

let empty = {
  instructions = [||];
  a = 0;
  b = 0;
  c = 0;
  d = 0;
  ip = Address 0;
  zero = false;
  carry = false;
  stack = [];
}

let init seznam_ukazov =
  let empty = {
    instructions = [||];
    a = 0;
    b = 0;
    c = 0;
    d = 0;
    ip = Address 0;
    zero = false;
    carry = false;
    stack = [];
  }
  in
  {empty with instructions = Array.of_list seznam_ukazov}

type register = A | B | C | D


type expression = Register of register | Const of int


type address = Address of int


type instruction =
    MOV of register * expression
  | ADD of register * expression
  | SUB of register * expression
  | INC of register
  | DEC of register
  | MUL of expression
  | DIV of expression
  | AND of register * expression
  | OR of register * expression
  | XOR of register * expression
  | NOT of register
  | CMP of register * expression
  | JMP of address
  | JA of address
  | JAE of address
  | JB of address
  | JBE of address
  | JE of address
  | JNE of address
  | CALL of address
  | RET
  | PUSH of expression
  | POP of register
  | HLT


type state = {
  instructions : instruction array;
  a : int;
  b : int;
  c : int;
  d : int;
  ip : address;
  zero : bool;
  carry : bool;
  stack : int list;
}


val empty : state =
  {instructions = [||]; a = 0; b = 0; c = 0; d = 0; ip = Address 0;
   zero = false; carry = false; stack = []}


val init : instruction list -> state = <fun>


In [175]:
let fibonacci n = [
  JMP (Address 20);       (* JMP main *)

(* fib: *)
  (* ; Shranimo vrednosti registrov *)
  PUSH (Register C);      (* PUSH C *)
  PUSH (Register B);      (* PUSH B *)

  (* ; V C shranimo začetno vrednost A *)
  MOV (C, Register A);    (* MOV C, A *)

  (* ; Če je A = 0, je to tudi rezultat *)
  CMP (A, Const 0);       (* CMP A, 0 *)
  JE (Address 17);        (* JE .fib_end *)

  (* ; Če je A = 1, je to tudi rezultat *)
  CMP (A, Const 1);       (* CMP A, 1 *)
  JE (Address 17);        (* JE .fib_end *)

  (* ; V nasprotnem primeru najprej izračunamo fib(A - 1) in ga shranimo v B *)
  DEC C;                  (* DEC C *)
  MOV (A, Register C);    (* MOV A, C *)
  CALL (Address 1);       (* CALL fib *)
  MOV (B, Register A);    (* MOV B, A *)

  (* ; Nato izračunamo še fib(A - 2) in ga shranimo v A *)
  DEC C;                  (* DEC C *)
  MOV (A, Register C);    (* MOV A, C *)
  CALL (Address 1);       (* CALL fib *)
  
  (* ; Nazadnje k A prištejemo še B, s čimer dobimo končni rezultat *)
  ADD (A, Register B);    (* ADD A, B *)
  JMP (Address 17);       (* JMP .fib_end *)

(* .fib_end: *)
  (* ; Povrnemo vrednosti registrov in vrnemo rezultat *)
  POP B;                  (* POP B *)
  POP C;                  (* POP C *)
  RET;                    (* RET *)

(* main: *)
  MOV (A, Const n);       (* MOV A, n *)
  CALL (Address 1);       (* CALL fib *)
  HLT;                    (* HLT *)
]

let fibonacci = {|
  JMP main
  ; Funkcija, ki izračuna fib(A) in vrednost shrani v register A
  fib:
      ; Shranimo vrednosti registrov
      PUSH C
      PUSH B
  
      ; V C shranimo začetno vrednost A
      MOV C, A
  
      ; Če je A = 0, je to tudi rezultat
      CMP A, 0
      JE .fib_end
  
      ; Če je A = 1, je to tudi rezultat
      CMP A, 1
      JE .fib_end
  
      ; V nasprotnem primeru najprej izračunamo fib(A - 1) in ga shranimo v B
      DEC C
      MOV A, C
      CALL fib
      MOV B, A
  
      ; Nato izračunamo še fib(A - 2) in ga shranimo v A
      DEC C
      MOV A, C
      CALL fib
      
      ; Nazadnje k A prištejemo še B, s čimer dobimo končni rezultat
      ADD A, B
      JMP .fib_end
  
  .fib_end:
      ; Povrnemo vrednosti registrov in vrnemo rezultat
      POP B
      POP C
      RET
  
  main:
      MOV A, 7
      CALL fib
  HLT
|}

val fibonacci : int -> instruction list = <fun>


val fibonacci : string =
  "\n  JMP main\n  ; Funkcija, ki izračuna fib(A) in vrednost shrani v register A\n  fib:\n      ; Shranimo vrednosti registrov\n      PUSH C\n      PUSH B\n  \n      ; V C shranimo začetno vrednost A\n      MOV C, A\n  \n      ; Če je A = 0, je to tudi rezultat\n      CMP A, 0\n      JE .fib_end\n  \n      ; Če"... (* string length 872; truncated *)


In [176]:
let read_instruction stanje_pomnilnika = 
  let st_ukaza = 
    match stanje_pomnilnika.ip with
    | Address x -> x
  in
  let tabela_ukazov = stanje_pomnilnika.instructions in
  if st_ukaza < Array.length tabela_ukazov then Some tabela_ukazov.(st_ukaza) else None


let read_register stanje_pomnilnika =
  function
  | A -> stanje_pomnilnika.a 
  | B -> stanje_pomnilnika.b 
  | C -> stanje_pomnilnika.c 
  | D -> stanje_pomnilnika.d 

let read_expression stanje_pomnilnika = 
  function
  | Register x -> read_register stanje_pomnilnika x
  | Const x -> x 

let write_register stanje_pomnilnika register vrednost = 
  match register with
  | A -> {stanje_pomnilnika with a = vrednost}
  | B -> {stanje_pomnilnika with b = vrednost}
  | C -> {stanje_pomnilnika with c = vrednost}
  | D -> {stanje_pomnilnika with d = vrednost}


let perform_unop f stanje_pomnilnika register = 
  write_register stanje_pomnilnika register (f (read_register stanje_pomnilnika register))


let perform_binop f stanje_pomnilnika register izraz = 
  let element_v_funkciji = 
    match izraz with 
    | Register x -> read_register stanje_pomnilnika x
    | Const x ->  x
  in
  write_register stanje_pomnilnika register (f (read_register stanje_pomnilnika register) element_v_funkciji)


let next = 
  function 
  | Address x -> Address (x + 1)

let jump stanje_pomnilnika naslov =
  {stanje_pomnilnika with ip = naslov}

let proceed stanje_pomnilnika = 
  {stanje_pomnilnika with ip = next stanje_pomnilnika.ip}


let push_stack stanje_pomnilnika stevilo =
  {stanje_pomnilnika with stack = stevilo :: stanje_pomnilnika.stack}

let pop_stack stanje_pomnilnika =
  match stanje_pomnilnika.stack with
  | [] -> failwith "Sklad je prazen"
  | x :: xs -> (x, {stanje_pomnilnika with stack = xs})

let compare stanje_pomnilnika (stevilo1: int) (stevilo2: int) = 
  let nov_zero = (stevilo1 = stevilo2) in
  let nov_carry = (stevilo1 < stevilo2) in
  {stanje_pomnilnika with zero = nov_zero; carry = nov_carry}

let conditional_jump stanje_pomnilnika naslov vrednost =
  match vrednost with
  | true -> jump stanje_pomnilnika naslov
  | false -> proceed stanje_pomnilnika

let call stanje_pomnilnika naslov = 
  match next stanje_pomnilnika.ip with
  | Address x -> push_stack (jump stanje_pomnilnika naslov) x

let return stanje_pomnilnika = 
  let naslov = fst (pop_stack stanje_pomnilnika) in
  let novo_stanje_pomnilnika = snd (pop_stack stanje_pomnilnika) in
  {novo_stanje_pomnilnika with ip = (Address naslov)}


let run_instruction st = function
  | MOV (reg, exp) -> write_register st reg (read_expression st exp) |> proceed
  | ADD (reg, exp) -> perform_binop ( + ) st reg exp |> proceed
  | SUB (reg, exp) -> if (read_register st reg) < (read_expression st exp) then failwith "Število ne sme biti negativno" 
                      else perform_binop ( - ) st reg exp |> proceed
  | INC reg -> perform_unop succ st reg |> proceed
  | DEC reg -> perform_unop (fun x -> if x = 0 then failwith "Število ne sme biti negativno" else x - 1) st reg |> proceed 
  | MUL exp -> perform_unop (fun x -> x * (read_expression st exp)) st A |> proceed
  | DIV exp -> perform_unop (fun x -> x / (read_expression st exp)) st A |> proceed
  (* Pozor, OCaml land/lor/lxor interpretira kot simbole, zato jih pišemo infiksno! *)
  | AND (reg, exp) -> perform_binop ( land ) st reg exp |> proceed
  | OR (reg, exp) -> perform_binop ( lor ) st reg exp |> proceed
  | XOR (reg, exp) -> perform_binop ( lxor ) st reg exp |> proceed
  | NOT reg -> perform_unop lnot st reg |> proceed
  | CMP (reg, exp) -> compare st (read_register st reg) (read_expression st exp) |> proceed
  | JMP add -> jump st add 
  | JA add -> conditional_jump st add (not st.carry && not st.zero)
  | JAE add -> conditional_jump st add (not st.carry) 
  | JB add -> conditional_jump st add (st.carry && not st.zero)
  | JBE add -> conditional_jump st add st.carry 
  | JE add -> conditional_jump st add st.zero 
  | JNE add -> conditional_jump st add (not st.zero) 
  | CALL add -> call st add 
  | RET -> return st 
  | PUSH exp -> push_stack st (read_expression st exp) |> proceed
  | POP reg ->
      let n, st' = pop_stack st in
      write_register st' reg n |> proceed
  | HLT -> failwith "Cannot execute instruction"



let rec run_program stanje_pomnilnika =
  let ukaz = read_instruction stanje_pomnilnika in
  match ukaz with
  | Some HLT -> stanje_pomnilnika
  | Some x -> run_program (run_instruction stanje_pomnilnika x)
  | None -> stanje_pomnilnika


(*NIZIIII*)
let parse_register niz : register = 
  match niz with
  | "A" -> A 
  | "B" -> B 
  | "C" -> C 
  | "D" -> D 
  | _ -> failwith "To ni register"

let parse_expression niz : expression = 
  match niz with 
  | "A" -> Register A 
  | "B" -> Register B 
  | "C" -> Register C 
  | "D" -> Register D 
  | drugo -> 
    match int_of_string_opt drugo with
    | Some st -> Const st
    | None -> failwith "Izraz ni niti register niti konstanta"



let clean_line niz =
  match String.index_opt niz ';' with
  | Some i -> String.trim (String.sub niz 0 i)
  | None -> String.trim niz

let clean_lines seznam_vrstic = 
  let rec pomozna_clean_lines stare_vrstice nove_vrstice = 
    match stare_vrstice with
    | [] -> List.rev nove_vrstice
    | x :: xs -> 
      match clean_line x with
      | "" -> pomozna_clean_lines xs nove_vrstice
      | vrstica -> pomozna_clean_lines xs (vrstica :: nove_vrstice)
    in
    pomozna_clean_lines seznam_vrstic []

  
let parse_address (seznam_nizov_in_naslovov: (string * address) list) (niz: string) = 
  let rec pomozna_parse_address sez_nizov = 
    match sez_nizov with
    | [] -> Address (int_of_string niz)
    | (ime, naslov) :: xs -> if ime = niz then naslov
      else pomozna_parse_address xs
    in pomozna_parse_address seznam_nizov_in_naslovov


let parse_label niz =
  let indeks_zadnjega_znaka = String.length niz - 1 
  in 
  match String.get niz indeks_zadnjega_znaka with
  | ':' -> Some (String.sub niz 0 indeks_zadnjega_znaka)
  | _ -> None


let parse_labels seznam_nizov =
  let rec pomozna_parse_labels vsi_nizi seznam_ukazov_z_naslovi nespremenjene_vrstice st_naslova = 
    match vsi_nizi with
    | [] -> (seznam_ukazov_z_naslovi, List.rev nespremenjene_vrstice)
    | x :: xs ->
      match parse_label x with 
      | Some ukaz -> pomozna_parse_labels xs ((ukaz, Address st_naslova) :: seznam_ukazov_z_naslovi) nespremenjene_vrstice st_naslova
      | None -> pomozna_parse_labels xs seznam_ukazov_z_naslovi (x :: nespremenjene_vrstice) (st_naslova + 1)
    in pomozna_parse_labels seznam_nizov [] [] 0


let parse_instruction labels line =
  let tokens =
    line
    |> String.split_on_char ' '
    |> List.concat_map (String.split_on_char ',')
    |> List.map String.trim
    |> List.filter (fun token -> token <> "")
  in
  match tokens with
  | ["MOV"; reg; exp] -> MOV (parse_register reg, parse_expression exp)
  | ["ADD"; reg; exp] -> ADD (parse_register reg, parse_expression exp)
  | ["SUB"; reg; exp] -> SUB (parse_register reg, parse_expression exp)
  | ["INC"; reg] -> INC (parse_register reg)
  | ["DEC"; reg] -> DEC (parse_register reg)
  | ["MUL"; exp] -> MUL (parse_expression exp)
  | ["DIV"; exp] -> DIV (parse_expression exp)
  | ["AND"; reg; exp] -> AND (parse_register reg, parse_expression exp)
  | ["OR"; reg; exp] -> OR (parse_register reg, parse_expression exp)
  | ["XOR"; reg; exp] -> XOR (parse_register reg, parse_expression exp)
  | ["NOT"; reg] -> NOT (parse_register reg)
  | ["CMP"; reg; exp] -> CMP (parse_register reg, parse_expression exp)
  | ["JMP"; add] -> JMP (parse_address labels add)
  | ["JA"; add] -> JA (parse_address labels add)
  | ["JAE"; add] -> JAE (parse_address labels add)
  | ["JB"; add] -> JB (parse_address labels add)
  | ["JBE"; add] -> JBE (parse_address labels add)
  | ["JE"; add] -> JE (parse_address labels add)
  | ["JNE"; add] -> JNE (parse_address labels add)
  | ["CALL"; add] -> CALL (parse_address labels add)
  | ["RET"] -> RET
  | ["PUSH"; exp] -> PUSH (parse_expression exp)
  | ["POP"; reg] -> POP (parse_register reg)
  | ["HLT"] -> HLT
  | _ -> failwith ("Invalid instruction: " ^ line)


val read_instruction : state -> instruction option = <fun>


val read_register : state -> register -> int = <fun>


val read_expression : state -> expression -> int = <fun>


val write_register : state -> register -> int -> state = <fun>


val perform_unop : (int -> int) -> state -> register -> state = <fun>


...

val clean_lines : string list -> string list = <fun>


val parse_address : (string * address) list -> string -> address = <fun>


val parse_label : string -> string option = <fun>


val parse_labels : string list -> (string * address) list * string list =
  <fun>


val parse_instruction : (string * address) list -> string -> instruction =
  <fun>


In [177]:
let run niz_z_ukazi =
  let seznam_vrstic = clean_lines (String.split_on_char '\n' niz_z_ukazi) in
  let seznam_navodil = parse_labels seznam_vrstic in
  let (oznake, ukazi) = seznam_navodil in
  let stanje_pomnilnika = init (List.map (parse_instruction oznake) ukazi) in
  run_program stanje_pomnilnika


val run : string -> state = <fun>


In [178]:
let primer_branje_11 =
  run fibonacci

val primer_branje_11 : state =
  {instructions =
    [|JMP (Address 20); PUSH (Register C); PUSH (Register B);
      MOV (C, Register A); CMP (A, Const 0); JE (Address 17);
      CMP (A, Const 1); JE (Address 17); DEC C; MOV (A, Register C);
      CALL (Address 1); MOV (B, Register A); DEC C; MOV (A, Register C);
      CALL (Address 1); ADD (A, Register B); JMP (Address 17); POP B; 
      POP C; RET; MOV (A, Const 7); CALL (Address 1); HLT|];
   a = 13; b = 0; c = 0; d = 0; ip = Address 22; zero = true; carry = false;
   stack = []}
